In [2]:
import pandas as pd
import os
import json
from pyecharts.charts import Graph
from pyecharts import options as opts
from pyecharts.globals import ThemeType
import re
import difflib
from pyecharts.commons.utils import JsCode
from bs4 import BeautifulSoup as bs
import base64
from ctypes import c_uint
import binascii

In [3]:
quest_map = {}
quest_line_map = {}
quest_id_set = set()
line_id_set = set()
quest_zh_map = {}
quest_zh_line_map = {}
with open("lang/zh_CN.lang", "r", encoding="utf-8") as f:
    for line in f.readlines():
        if "betterquesting.questline" in line and "=" in line:
            one_line = line.split("=")
            quest_line_id = one_line[0].split(".")[-2]
            if "name" in one_line[0]:
                quest_zh_line_map[quest_line_id + "name"] = one_line[1]
            elif "desc" in one_line[0]:
                quest_zh_line_map[quest_line_id + "desc"] = one_line[1]
            line_id_set.add(quest_line_id)
        elif "betterquesting.quest" in line and "=" in line:
            one_line = line.split("=", -1)
            quest_id = one_line[0].split(".")[-2]
            if "name" in one_line[0]:
                quest_zh_map[quest_id + "name"] = one_line[1]
            elif "desc" in one_line[0]:
                quest_zh_map[quest_id + "desc"] = one_line[1]
            quest_id_set.add(quest_id)

quest_name_df = pd.DataFrame(columns=["id", "name", "desc", "zh_name", "zh_desc"])
for i in quest_id_set:
    name = quest_map[i + "name"].strip() if (i + "name") in quest_map else ""
    desc = quest_map[i + "desc"].strip() if (i + "desc") in quest_map else ""
    zh_name = quest_zh_map[i + "name"].strip() if (i +
                                                   "name") in quest_zh_map else ""
    zh_desc = quest_zh_map[i + "desc"].strip()if (i +
                                                  "desc") in quest_zh_map else ""
    quest_name_df.loc[len(quest_name_df)] = [i.replace(
        "quest", ""), name, desc, zh_name, zh_desc]

quest_line_name = pd.DataFrame(
    columns=["id", "name", "desc", "zh_name", "zh_desc"])
for i in line_id_set:
    name = quest_line_map[i + "name"].strip() if (i +
                                                  "name") in quest_line_map else ""
    desc = quest_line_map[i + "desc"].strip() if (i +
                                                  "desc") in quest_line_map else ""
    zh_name = quest_zh_line_map[i +
                                "name"].strip() if (i + "name") in quest_zh_line_map else ""
    zh_desc = quest_zh_line_map[i +
                                "desc"].strip()if (i + "desc") in quest_zh_line_map else ""
    quest_line_name.loc[len(quest_line_name)] = [i.replace(
        "line", ""), name, desc, zh_name, zh_desc]

In [4]:
df = pd.DataFrame(columns=["quest_file_name", "quest_id",
                  "quest_name", "quest_desc", "quest_icon", "pre_quests", "quest_name_with_format", "is_main"])
for quest_line in os.listdir("DefaultQuests/Quests/"):
    for quest in os.listdir(f"DefaultQuests/Quests/{quest_line}/"):
        que = json.load(
            open(f"DefaultQuests/Quests/{quest_line}/{quest}", "r", encoding="utf-8"))
        
        quest_high_id = que["questIDHigh:4"] if "questIDHigh:4" in que else 0
        quest_low_id = que["questIDLow:4"]
        quest_name = que["properties:10"]["betterquesting:10"]["name:8"]
        quest_desc = que["properties:10"]["betterquesting:10"]["desc:8"]
        icon_name = "quests_icons/QuestIcon/" + str(quest_low_id) + ".png"
        is_main = que["properties:10"]["betterquesting:10"]["isMain:1"]
        
        pre_quests = []
        for i in que["preRequisites:9"]:
            pre_quests.append(que["preRequisites:9"][i]["questIDLow:4"])

        # if f"quest{quest_id}name" in quest_zh_map:
        #     quest_name = quest_zh_map[f"quest{quest_id}name"]
        # if f"quest{quest_id}desc" in quest_zh_map:
        #     quest_desc = quest_zh_map[f"quest{quest_id}desc"]
        
        str_id = base64.urlsafe_b64encode(bytes.fromhex((quest_high_id.to_bytes(length=8, signed=True, byteorder="big") +
                                          quest_low_id.to_bytes(length=8, signed=True, byteorder="big")).hex())).decode("utf-8").replace("==", "")
        line = quest_name_df[quest_name_df["id"] == str_id]
        if len(line) != 0:
            quest_name = line["zh_name"].values[0]
            quest_desc = line["zh_desc"].values[0]

        # quest_name = re.sub("(§.)", "", quest_name.strip())
        # quest_desc = re.sub("(§.)", "", quest_desc.strip())

        quest_desc = quest_desc.replace("%n", "</br>")

        df.loc[len(df)] = [quest, quest_low_id, re.sub("(§.)", "", quest_name.strip()),
                           quest_desc, icon_name, pre_quests, quest_name, is_main]


repeats = list(set(df[df["quest_name"].duplicated()]["quest_name"]))
for word in repeats:
    counter = 1
    for i in df[df["quest_name"] == word].sort_values("quest_id").index:
        df.loc[i, "quest_name"] = df.loc[i, "quest_name"] + str(counter)
        counter += 1

In [26]:
if_json = True
whole_json = json.loads("{}")

for questline in os.listdir("DefaultQuests/QuestLines"):
    nodes = []
    links = []
    max_size = 0
    node_size = []
    fake_index = 0
    for quest in os.listdir(f"DefaultQuests/QuestLines/{questline}"):
        if quest == "QuestLine.json":
            continue

        que = json.load(
            open(f"DefaultQuests/QuestLines/{questline}/{quest}", "r", encoding="utf-8"))
        line = df[df["quest_id"] == que["questIDLow:4"]]

        desc = line["quest_desc"].values[0]

        temp_list = []
        for i in range(0, len(desc), 50):
            temp_list.append(desc[i : i + 50])

        path = f"image://version/272/{line['quest_icon'].values[0]}"

        name = line["quest_name"].values[0]

        if "questIDHigh:4" in que:
            high_id = que["questIDHigh:4"]
        else:
            high_id = 0
        low_id = que["questIDLow:4"]

        quest_id_str = base64.urlsafe_b64encode(bytes.fromhex((high_id.to_bytes(
            length=8, signed=True, byteorder="big") + low_id.to_bytes(length=8, signed=True, byteorder="big")).hex())).decode("utf-8")

        title = line["quest_name_with_format"].values[0]

        is_main = line["is_main"].values[0]

        new_node = {"name": name,
                    "symbolSize": int(que["sizeX:3"]),
                    "symbol": path,
                    "x": (que["x:3"] + int(que["sizeX:3"]) / 2),
                    "y": (que["y:3"] + int(que["sizeX:3"]) / 2),
                    "data": desc,
                    "quest_id": quest_id_str,
                    "title": title,
                    "emphasis": {
                        "disabled": True,
                        "scale": 1,
                        "focus": "None",
                    },
                    }
        
        fake_node = {
            "name": fake_index,
            "symbolSize": int(que["sizeX:3"] * 1.3),
            "symbol": f"image://static/{"main" if is_main else "not_main"}_unclaim.png",
            "x": (que["x:3"] + int(que["sizeX:3"]) / 2),
            "y": (que["y:3"] + int(que["sizeX:3"]) / 2),
            "emphasis":{
                "disabled": True,
            },
            "select":{
                "disabled": True,
            },
        }

        node_size.append(fake_node["symbolSize"])

        nodes.append(fake_node)
        nodes.append(new_node)
        fake_index += 1
        pre_quest_names = []
        pre_quests = line["pre_quests"].values[0]
        for quest_id in pre_quests:
            pre_que = df[df["quest_id"] == quest_id]
            pre_name = pre_que["quest_name"].values[0]
            pre_quest_names.append(pre_name)

            temp_link = {"source": pre_name,
                         "target": name,
                         "symbol": ["none", "arrow"]}
            if int(pre_que["is_main"].iloc[0]) == int(is_main) == 1:
                temp_link["lineStyle"] = {"color": "#00c800", "width":3}
            links.append(temp_link)
            
        new_node["tooltip"] = "<strong>" + name + \
            "</strong></br>前置任务需求:</br>" + "</br>".join(pre_quest_names)

    avg = sum(node_size) / len(node_size)
    multi = 40 / avg
    for node in nodes:
        node["symbolSize"] = int(multi * node["symbolSize"])


    g = (
        Graph(init_opts=opts.InitOpts(width="100%",
              height="900%", bg_color="#f5f0d3", chart_id="this_chart", ))
        .add("", nodes, links, is_draggable=False, edge_symbol=['circle', 'arrow'], repulsion=0, label_opts=opts.LabelOpts(position="up", is_show=False), gravity=0, layout="none", edge_symbol_size=10)
    )
    if if_json:
        json_str = g.dump_options_with_quotes()
        series = json.loads(json_str)["series"][0]
        whole_json[questline.split('-')[0]] = {"data": series["data"],"links": series["links"]}
    else:
        g.render(f"html/{questline.split('-')[0]}.html")

if if_json:
    whole_json["selectedMode"] = True
    with open(f"json/quest_json.json", "w", encoding="utf-8") as quest_json:
        json.dump(whole_json, quest_json, indent=4, separators=(',', ':'), ensure_ascii=False)
else:
    for html in os.listdir("html"):
        if not "html" in html:
            continue
        with open(f"html/{html}", encoding="utf-8") as page:
            a = bs(page)
        os.remove(f"html/{html}")
        a = a.find_all("script")[1]
        with open(f"js/{html.replace('.html', '.js')}", "w", encoding="utf-8") as js:
            js.write(a.text)

In [ ]:
with open("js/test.json", "w", encoding="utf-8") as f:
    f.write(g)

In [61]:
quest_line_order = []
for line in open("DefaultQuests/QuestLinesOrder.txt").readlines():
    quest_line_order.append(line.split(": ")[1].strip())

In [6]:
counter = 0
with open("lang/en_US.lang", "r", encoding="utf-8") as f:
    for line in f.readlines():
        if "=" in line:
            counter += 1
counter

6626

In [7]:
counter = 0
with open("lang/zh_CN.lang", "r", encoding="utf-8") as f:
    for line in f.readlines():
        if "=" in line:
            counter += 1
counter

6684

In [90]:
int(base64.b64decode("LIZCdBLfQquQpULGmFDixw==".encode("ascii")).hex(), 16)

59183147428921293450463928407402013383

In [79]:
base64.b64encode(bytes.fromhex("0" * (32-3) + hex(260)[2:])).decode("utf-8")

'AAAAAAAAAAAAAAAAAAABBA=='

In [314]:
base64.b64encode(bytes.fromhex((que["questIDHigh:4"].to_bytes(length=8, signed=True) +
                                que["questIDLow:4"].to_bytes(length=8, signed=True)).hex())).decode("utf-8")

'LIZCdBLfQquQpULGmFDixw=='

In [238]:
"LIZCdBLfQquQpULGmFDixw=="

'LIZCdBLfQquQpULGmFDixw=='

'0x9850e2c7'

In [341]:
temp_str = "4096kEssentiaSto-G5g_5_24RS6sqhmjFEbf8A==.json".split("-", 1)[
    1].replace(".json", "")
re.sub("[^a-zA-Z0-9/-/_]", "", temp_str)

'G5g_5_24RS6sqhmjFEbf8A'

In [149]:
temp_str = "欢迎来到TecTech!这里有你毕业所需的真正的巅峰科技.但是,学习TecTech常常让人摸不着头脑,所以仔细听.%n%n首先你需要制造所需机器的多种机械方块:超能机械方块,电子计算机机械方块,高级计算机方块和计算机散热器.你需要大量这些机械方块才能继续前进.%n%n我们列举出来的只是举个例子,实际需要的只会更多、更多、更多.%n%n使用光缆将机器连接起来. 确保它们已被涂色,同时光缆连接到的方块也要被涂色.%n%n*投影仪将会显示完整机器的全息图像,左下角的聊天栏会显示可变方块的注释. 如果堆叠更多投影仪的话,你就可以看到较大版本的全息图(比如量子计算机).%n%n§4译者注:§lTecTech多方块机器的控制器GUI界面的闪电或是电源图标亮起时,不要破坏任何结构方块,否则会引发爆炸!运行过程中更不能破坏结构方块!§r"

In [150]:
temp_q = ""
flag = False
code = []
for i in re.split("(§.)", temp_str):
    if len(i) == 0:
        continue
    if "§" in i:
        flag = True
        if "r" in i:
            flag = False
            print(temp_q)
            print(code)
            code = []
        else:
            code.append(i)
    else:
        if flag is False:
            print(i)
        else:
            temp_q += i

欢迎来到TecTech!这里有你毕业所需的真正的巅峰科技.但是,学习TecTech常常让人摸不着头脑,所以仔细听.%n%n首先你需要制造所需机器的多种机械方块:超能机械方块,电子计算机机械方块,高级计算机方块和计算机散热器.你需要大量这些机械方块才能继续前进.%n%n我们列举出来的只是举个例子,实际需要的只会更多、更多、更多.%n%n使用光缆将机器连接起来. 确保它们已被涂色,同时光缆连接到的方块也要被涂色.%n%n*投影仪将会显示完整机器的全息图像,左下角的聊天栏会显示可变方块的注释. 如果堆叠更多投影仪的话,你就可以看到较大版本的全息图(比如量子计算机).%n%n
译者注:TecTech多方块机器的控制器GUI界面的闪电或是电源图标亮起时,不要破坏任何结构方块,否则会引发爆炸!运行过程中更不能破坏结构方块!
['§4', '§l']


In [151]:
re.split("(§.)", temp_str)

['欢迎来到TecTech!这里有你毕业所需的真正的巅峰科技.但是,学习TecTech常常让人摸不着头脑,所以仔细听.%n%n首先你需要制造所需机器的多种机械方块:超能机械方块,电子计算机机械方块,高级计算机方块和计算机散热器.你需要大量这些机械方块才能继续前进.%n%n我们列举出来的只是举个例子,实际需要的只会更多、更多、更多.%n%n使用光缆将机器连接起来. 确保它们已被涂色,同时光缆连接到的方块也要被涂色.%n%n*投影仪将会显示完整机器的全息图像,左下角的聊天栏会显示可变方块的注释. 如果堆叠更多投影仪的话,你就可以看到较大版本的全息图(比如量子计算机).%n%n',
 '§4',
 '译者注:',
 '§l',
 'TecTech多方块机器的控制器GUI界面的闪电或是电源图标亮起时,不要破坏任何结构方块,否则会引发爆炸!运行过程中更不能破坏结构方块!',
 '§r',
 '']

In [57]:
def process(temp_str: str, type: list[str]):
    ...

IndexError: no such group